Intro stuff

In [23]:
import tweepy
import json
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
nlp = spacy.load("en_core_web_sm")
consumer_key = "U6eIfx0myrDZxymjY6rxFQuH3"
consumer_secret = "bmHWK7d1ktxIa43iOar1eyPs2kG9frLFecv0E6mdYWXex3VCnM"
access_token = "1351114116291842050-Sqx8kbMr7mVneQasWdjkrj9CH4NfI8"
access_token_secret = "VfmTCgDjmPRAQ7vwGMysI6izKnvgPSrRqqO035ZPxDIPT"

In [24]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.keys())
    


dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])


Get first tweets from topic

### main function

In [13]:
data = api.search("tjuvjakt -filter:retweets", count = 100, lang = "sv")
data

{'statuses': [{'created_at': 'Tue Mar 02 08:07:12 +0000 2021',
   'id': 1366661352031481858,
   'id_str': '1366661352031481858',
   'text': 'Hej #polisen! Djurrätt är en åsikt. Självklart ska man inte ofreda, förstöra eller förtala. Men jag utgår ifrån att… https://t.co/Cs5uNxlHSr',
   'truncated': True,
   'entities': {'hashtags': [{'text': 'polisen', 'indices': [4, 12]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/Cs5uNxlHSr',
      'expanded_url': 'https://twitter.com/i/web/status/1366661352031481858',
      'display_url': 'twitter.com/i/web/status/1…',
      'indices': [117, 140]}]},
   'metadata': {'iso_language_code': 'sv', 'result_type': 'recent'},
   'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
   'in_reply_to_status_id': None,
   'in_reply_to_status_id_str': None,
   'in_reply_to_user_id': None,
   'in_reply_to_user_id_str': None,
   'in_reply_to_screen_name': None,
   'user': {'id': 19492635,
    'id_

In [25]:
def get_data(query, topics, lang):
    data = api.search(query + " movie -filter:retweets", count = 100, lang = lang)
    tweets = list()
    clean_tweets = list()
    blacklist = query.split()
    blacklist.append("movie")
    
    #get tweets
    for tweet in data["statuses"]:
        tweets.append(tweet.get("text")) 
        
    #clean tweets
    for tweet in tweets:   
        doc = nlp(tweet)
        clean_tweet = ""
        for token in doc:
            if token.is_alpha and not token.is_stop and len(token.text) > 3 and token.lower_ not in blacklist:
                clean_tweet += token.lower_ + " "
        clean_tweets.append(clean_tweet)     
    d = {"text" : clean_tweets}
    df = pd.DataFrame(data=d)
    
    #tdif data and cluster with LDA 
    vect = TfidfVectorizer()
    tweet_doc = vect.fit_transform(df["text"])

    LDA = LatentDirichletAllocation(n_components=topics, random_state=10)
    LDA.fit(tweet_doc)
    
    # generate topic lists
    topics = list()
    for i,topic in enumerate(LDA.components_):
        topics.append([vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    
    return topics

#### Run model

In [27]:
topics = get_data("uncut gems", 3, "en")
for i,topic in enumerate(topics):
    print("Topic",i,":")
    print(topic)

Topic 0 :
['came', 'masterpiece', 'montage', 'need', 'howard', 'like', 'watch', 'furby', 'feel', 'netflix']
Topic 1 :
['julia', 'goated', 'trailer', 'good', 'watching', 'watched', 'movies', 'seen', 'watch', 'time']
Topic 2 :
['truly', 'like', 'crazy', 'said', 'scene', 'good', 'energy', 'adam', 'sandler', 'care']


<!-- ### Store in csv files or similar  -->